
# MVP: *Machine Learning & Analytics* — Breast Cancer (Classificação)
**Autor:** _CAIO CARDOSO DE SOUZA_  
**Data:** 15/09/2025  
**Matrícula:** 4052025000946  
**Dataset:** [Breast Cancer Wisconsin (Diagnostic) — scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset)

---



## 1. Escopo, objetivo e definição do problema

**Contexto:** O câncer de mama é uma das principais causas de mortalidade entre mulheres. A detecção precoce aumenta a chance de tratamento eficaz e reduz custos.  
**Objetivo:** Construir e comparar modelos de *machine learning* para classificar tumores como **malignos (0)** ou **benignos (1)** a partir de 30 variáveis numéricas extraídas de imagens histopatológicas.  
**Tipo de tarefa:** Classificação binária supervisionada.  
**Área:** Dados tabulares biomédicos.  
**Valor de negócio/usuário:** Apoio à decisão clínica (triagem). *Este notebook é educacional e não substitui diagnóstico médico.*


## 2. Reprodutibilidade e ambiente

In [ ]:

# === Bloco 2: Setup básico e reprodutibilidade ===
import warnings; warnings.filterwarnings("ignore")

import sys, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn core
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import (train_test_split, StratifiedKFold, cross_val_score,
                                     GridSearchCV)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# modelos
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import (BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier,
                              AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier)

# métricas/plots
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score, classification_report,
                             ConfusionMatrixDisplay)

# reprodutibilidade
SEED = 42
np.random.seed(SEED); random.seed(SEED)

# visualização de dataframes sem quebra
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print("Python:", sys.version.split()[0], "| Seed:", SEED)


### 2.2 Funções auxiliares

In [ ]:

def evaluate_classification(y_true, y_pred, proba=None):
    # Retorna dicionário com métricas principais
    out = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_weighted": f1_score(y_true, y_pred, average="weighted"),
        "roc_auc": roc_auc_score(y_true, proba[:,1]) if proba is not None else np.nan
    }
    return out

def summarize_cv(results_dict):
    # Imprime média e desvio dos scores de CV e retorna ranking
    lines = []
    for name, scores in results_dict.items():
        lines.append((name, scores.mean(), scores.std()))
    lines = sorted(lines, key=lambda t: t[1], reverse=True)
    for name, m, s in lines:
        print(f"{name}: {m:.3f} ({s:.3f})")
    return lines



## 3. Dados: carga, entendimento e qualidade

**Fonte:** *scikit-learn* (`load_breast_cancer`). 569 instâncias, 30 variáveis numéricas.  
**Ética/licença:** dataset público; sem dados pessoais identificáveis. Uso educacional.  
**Prevenção de *data leakage*:** divisão treino/teste **antes** de qualquer transformação; todas as transformações ajustadas **apenas no treino** via `Pipeline/ColumnTransformer`.


In [ ]:

# === Bloco 3: Carga e entendimento dos dados ===
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

df = pd.concat([X, y], axis=1)
print("Formato (linhas, colunas):", df.shape)
print("Distribuição de classes {maligno, benigno}:", dict(zip(data.target_names, np.bincount(data.target))))
df.head(10)


### 3.1 Análise exploratória resumida (EDA)

In [ ]:

desc = X.describe().T[["mean","std","min","max"]]
display(desc.head(8))

# distribuição da classe
_, counts = np.unique(y, return_counts=True)
plt.bar(["maligno (0)", "benigno (1)"], counts)
plt.title("Distribuição de classes"); plt.show()

# correlação rápida (subconjunto para visualização)
corr_subset = X.iloc[:, :12].corr()
plt.imshow(corr_subset, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar(); plt.title("Correlação (12 primeiras features)"); plt.show()


## 4. Definição do target, variáveis e divisão dos dados

In [ ]:

# === Bloco 4: Target e split 80/20 ===
target = "target"
features = X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X[features], y, test_size=0.20, stratify=y, random_state=SEED
)
print("Train:", X_train.shape, "| Test:", X_test.shape)
print("Proporção de classes (train):\n", y_train.value_counts(normalize=True).round(3))
print("Proporção de classes (test):\n", y_test.value_counts(normalize=True).round(3))



## 5. Seleção de variáveis (*Feature Selection*) — opcional, orientada por importância

**Por quê agora?** Após o *split*, antes de montar os *pipelines*. Assim garantimos que a seleção é aprendida **só** no treino, evitando *data leakage* e permitindo que os *preprocessors* usem apenas as colunas escolhidas.

**Critério adaptativo:**  
1) Ajustamos um **RandomForest** (robusto a escala) no treino com *imputação* para obter `feature_importances_`.  
2) Se a razão `max_importance / min_importance < 3`, entendemos que os "pesos" são relativamente parecidos → **mantemos todas** as 30 variáveis (preferência por interpretabilidade e simplicidade).  
3) Caso contrário, mantemos o menor conjunto de variáveis que acumule **≥95%** da importância total (com no mínimo 12 features), e reportamos a lista final.  
4) Como checagem, também calculamos o ranking univariado (`SelectKBest(f_classif)`) e mostramos a interseção com as escolhidas.


> **Nota didática:** embora este dataset seja pequeno (569 amostras, 30 variáveis),
incluímos a etapa de *feature selection* **por fins educacionais**, para documentar o processo,
mostrar como evitar *data leakage* e observar o impacto na validação cruzada.


In [ ]:

# === Bloco 5: Feature Selection (condicional) ===
from sklearn.feature_selection import SelectKBest, f_classif

# 1) RF só com imputação (sem escala)
num_cols_all = X_train.columns.tolist()
rf_fs = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(n_estimators=400, random_state=SEED, n_jobs=-1))
])
rf_fs.fit(X_train[num_cols_all], y_train)
importances = rf_fs.named_steps["rf"].feature_importances_
imp_ser = pd.Series(importances, index=num_cols_all).sort_values(ascending=False)

# Plot top-20 para diagnóstico
ax = imp_ser.head(20).plot(kind="barh", figsize=(8,6))
plt.gca().invert_yaxis()
plt.title("Importância de features (RF) — Top 20")
plt.show()

ratio = imp_ser.max() / max(imp_ser.min(), 1e-12)
cum = imp_ser.cumsum()

# 2) Decisão adaptativa
if ratio < 3.0:
    decision = "keep_all"
    final_num_cols = num_cols_all  # pesos semelhantes → manter todas
else:
    cutoff_idx = np.argmax(cum.values >= 0.95)  # menor índice que atinge 95%
    cutoff_idx = max(cutoff_idx, 11)            # garantir pelo menos 12 colunas
    final_num_cols = imp_ser.index[:cutoff_idx+1].tolist()
    decision = "select_subset"

print(f"Razão max/min de importância: {ratio:.2f}")
print(f"Decisão: {decision}")
print(f"Nº de colunas finais: {len(final_num_cols)} / {len(num_cols_all)}")

# 3) Checagem com SelectKBest (mesmo tamanho do conjunto escolhido)
k_sel = len(final_num_cols)
skb = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("std", StandardScaler()),
    ("skb", SelectKBest(score_func=f_classif, k=k_sel))
])
skb.fit(X_train[num_cols_all], y_train)
mask = skb.named_steps["skb"].get_support()
skb_cols = X_train.columns[mask].tolist()

inter = set(final_num_cols).intersection(skb_cols)
print(f"Interseção RF-Importância vs SelectKBest: {len(inter)} colunas")
print(sorted(list(inter))[:10], "...")

# Vamos registrar o racional em texto para o relatório
if decision == "keep_all":
    rationale_text = (
        "Os pesos relativos estão próximos (max/min < 3). Mantivemos as 30 variáveis "
        "para preservar interpretabilidade e evitar risco de remover sinais úteis; "
        "models baseados em árvore e regressão logística lidam bem com esse tamanho."
    )
else:
    rationale_text = (
        f"Os pesos variaram bastante (max/min = {ratio:.2f}). Mantivemos "
        f"{len(final_num_cols)} variáveis que acumulam ≥95% da importância (RF). "
        "Isso tende a reduzir variância e tempo de treino, com baixo custo de viés."
    )

rationale_text



### 5.1 Efeito da *feature selection*: antes vs. depois (CV estratificada 10-fold)

Para fins **didáticos**, comparamos rapidamente 3 modelos (Regressão Logística, RandomForest, KNN)
**antes** (todas as 30 variáveis) e **depois** (conjunto `final_num_cols`) da seleção,
sempre dentro de *pipelines* para evitar *leakage*. Reportamos média e desvio-padrão da **acurácia (CV=10)**
e o **delta** entre as versões.


In [ ]:

# === Bloco 5.1: Comparação antes/depois da FS (CV=10) ===
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv10 = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# Pipelines "antes" (todas as colunas)
all_cols = X_train.columns.tolist()
pre_none_all = ColumnTransformer([("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), all_cols)])
pre_std_all  = ColumnTransformer([("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), all_cols)])

models_before = {
    "LogReg": Pipeline([("pre", pre_std_all),  ("m", LogisticRegression(max_iter=1000, random_state=SEED))]),
    "RF"    : Pipeline([("pre", pre_none_all), ("m", RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1))]),
    "KNN"   : Pipeline([("pre", pre_std_all),  ("m", KNeighborsClassifier())])
}

# Pipelines "depois" (features finais)
pre_none_fs = ColumnTransformer([("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), final_num_cols)])
pre_std_fs  = ColumnTransformer([("num", Pipeline([("imp", SimpleImputer(strategy="median")), ("sc", StandardScaler())]), final_num_cols)])

models_after = {
    "LogReg": Pipeline([("pre", pre_std_fs),  ("m", LogisticRegression(max_iter=1000, random_state=SEED))]),
    "RF"    : Pipeline([("pre", pre_none_fs), ("m", RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1))]),
    "KNN"   : Pipeline([("pre", pre_std_fs),  ("m", KNeighborsClassifier())])
}

rows = []
for name in ["LogReg", "RF", "KNN"]:
    sc_before = cross_val_score(models_before[name], X_train, y_train, cv=cv10, scoring="accuracy", n_jobs=-1)
    sc_after  = cross_val_score(models_after[name],  X_train, y_train, cv=cv10, scoring="accuracy", n_jobs=-1)
    rows.append({
        "modelo": name,
        "antes_mean": sc_before.mean(), "antes_std": sc_before.std(),
        "depois_mean": sc_after.mean(), "depois_std": sc_after.std(),
        "delta_mean": sc_after.mean() - sc_before.mean()
    })

cmp_df = pd.DataFrame(rows).set_index("modelo").round(4)
print("Comparação (acurácia CV=10) — antes vs. depois da FS")
display(cmp_df)

# Visualização simples (matplotlib)
import matplotlib.pyplot as plt
ax = cmp_df[["antes_mean","depois_mean"]].plot(kind="bar", figsize=(8,4))
plt.title("Acurácia média (CV=10) — antes vs. depois da FS")
plt.ylabel("acurácia")
plt.xticks(rotation=0)
plt.legend()
plt.show()


## 6. Tratamento dos dados e Pipeline de pré-processamento

In [ ]:

# === Bloco 6: Pré-processamento com as features finais ===
num_cols = final_num_cols  # usar decisão do bloco anterior

numeric_pipe_none = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
numeric_pipe_std  = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),
                                    ("scaler", StandardScaler())])
numeric_pipe_min  = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),
                                    ("scaler", MinMaxScaler())])

preproc_none = ColumnTransformer([("num", numeric_pipe_none, num_cols)])
preproc_std  = ColumnTransformer([("num", numeric_pipe_std,  num_cols)])
preproc_min  = ColumnTransformer([("num", numeric_pipe_min,  num_cols)])

print(f"Colunas usadas nos pipelines: {len(num_cols)}")


In [ ]:

# === Bloco 5: Pré-processamento com ColumnTransformer ===
num_cols = X_train.columns.tolist()

numeric_pipe_none = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
])

numeric_pipe_std = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

numeric_pipe_min = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

preproc_none = ColumnTransformer([("num", numeric_pipe_none, num_cols)])
preproc_std  = ColumnTransformer([("num", numeric_pipe_std,  num_cols)])
preproc_min  = ColumnTransformer([("num", numeric_pipe_min,  num_cols)])


## 6. Baseline e modelos candidatos

In [ ]:

# === Bloco 6: Definição dos modelos e montagem das pipelines ===
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

models = {
    "LR": LogisticRegression(max_iter=1000, random_state=SEED),
    "KNN": KNeighborsClassifier(),
    "CART": DecisionTreeClassifier(random_state=SEED),
    "NB": GaussianNB(),
    "SVM": SVC(probability=True, random_state=SEED),
    "Bag": BaggingClassifier(random_state=SEED),
    "RF": RandomForestClassifier(n_estimators=200, random_state=SEED),
    "ET": ExtraTreesClassifier(n_estimators=200, random_state=SEED),
    "Ada": AdaBoostClassifier(n_estimators=150, random_state=SEED),
    "GB": GradientBoostingClassifier(n_estimators=200, random_state=SEED),
}

voting = VotingClassifier(estimators=[("lr", models["LR"]),
                                      ("rf", models["RF"]),
                                      ("gb", models["GB"])],
                          voting="soft", n_jobs=-1)

pipelines = {}
# baseline
pipelines["Dummy"] = Pipeline([("pre", preproc_none), ("model", DummyClassifier(strategy="most_frequent"))])

for tag, pre in [("orig", preproc_none), ("padr", preproc_std), ("norm", preproc_min)]:
    for mname, m in {**models, "Vot": voting}.items():
        pipelines[f"{mname}-{tag}"] = Pipeline([("pre", pre), ("model", m)])


### 6.1 Treino e avaliação rápida (CV + boxplot)

In [ ]:

cv_scores = {}
names, results = [], []

for name, pipe in pipelines.items():
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
    cv_scores[name] = scores
    names.append(name); results.append(scores)

_ = summarize_cv(cv_scores)

# Boxplot comparativo
plt.figure(figsize=(20,8))
plt.boxplot(results, labels=names)
plt.xticks(rotation=90)
plt.title("Comparação de Modelos (orig/padr/norm) - Acurácia (CV)")
plt.ylabel("Accuracy (CV)")
plt.show()


## 7. Validação e Otimização de Hiperparâmetros (GridSearch KNN)

In [ ]:

param_grid_knn = {
    "model__n_neighbors": list(range(1, 22, 2)),
    "model__metric": ["euclidean", "manhattan", "minkowski"]
}

gs_results = {}
for tag, pre in [("orig", preproc_none), ("padr", preproc_std), ("norm", preproc_min)]:
    pipe_knn = Pipeline([("pre", pre), ("model", KNeighborsClassifier())])
    gs = GridSearchCV(pipe_knn, param_grid_knn, scoring="accuracy", cv=cv, n_jobs=-1, verbose=0)
    gs.fit(X_train, y_train)
    gs_results[tag] = gs
    print(f"[KNN-{tag}] melhor score (CV): {gs.best_score_:.4f} | melhores params: {gs.best_params_}")


## 8. Avaliação final, análise de erros e limitações

In [ ]:

# Seleção do melhor por CV
best_name, best_mean = None, -np.inf
for name, scores in cv_scores.items():
    if scores.mean() > best_mean:
        best_name, best_mean = name, scores.mean()

best_pipe = pipelines[best_name]
print(f"Melhor por CV: {best_name} | mean={best_mean:.4f}")

# treino no conjunto de treino completo
best_pipe.fit(X_train, y_train)

# avaliação em teste
y_pred = best_pipe.predict(X_test)
y_proba = best_pipe.predict_proba(X_test) if hasattr(best_pipe, "predict_proba") else None

metrics = evaluate_classification(y_test, y_pred, y_proba)
print("\nMétricas (teste):", metrics)
print("\nClassification report:\n", classification_report(y_test, y_pred))

# matriz de confusão
ConfusionMatrixDisplay.from_estimator(best_pipe, X_test, y_test, display_labels=data.target_names)
plt.title(f"Matriz de confusão — {best_name} (teste)")
plt.show()

# importância de features se disponível
if hasattr(best_pipe.named_steps["model"], "feature_importances_"):
    importances = best_pipe.named_steps["model"].feature_importances_
    imp = pd.Series(importances, index=X.columns).sort_values(ascending=False).head(15)
    imp.plot(kind="barh"); plt.gca().invert_yaxis()
    plt.title("Top 15 importâncias (árvore/ensemble)"); plt.show()


## 9. Refit com todo o dataset e salvando artefatos

In [ ]:

# Refit no dataset inteiro (produção)
best_pipe.fit(X, y)

# (Opcional) salvar pipeline
# import joblib; joblib.dump(best_pipe, "breast_cancer_best_pipeline.joblib")


## 10. Predição em novos dados

In [ ]:

# Simulação de 3 novas instâncias
novos = X.sample(3, random_state=SEED).reset_index(drop=True)
pred = best_pipe.predict(novos)
pred_labels = [data.target_names[i] for i in pred]
print("Predições (0=maligno, 1=benigno):", pred.tolist(), "->", pred_labels)
